<a href="https://colab.research.google.com/github/pushpamohan/dynamictimetable/blob/master/Copy_of_Copy_of_bisect_l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# memory check -GPU

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=eeb738927d4986fe78ee6810d5be83eaa40dae4e52b8a05315072add22cb478e
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.9 GB  | Proc size: 154.2 MB
GPU RAM Free: 15079MB | Used: 0MB | Util   0% | Total 15079MB


## install pyspark



In [0]:
!pip install pyspark

     |████████████████████████████████| 215.6MB 139kB/s 
     |████████████████████████████████| 204kB 45.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.3-py2.py3-none-any.whl size=215964963 sha256=cd69d67b61dae747380a339816a56472b1e91c608b809e44a3337c4cdf8bc51c
  Stored in directory: /root/.cache/pip/wheels/8d/20/f0/b30e2024226dc112e256930dd2cd4f06d00ab053c86278dcf3
Successfully built pyspark


# **install jdk hadoop  spark**

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.3/spark-2.4.3-bin-hadoop2.7.tgz
!tar xf spark-2.4.3-bin-hadoop2.7.tgz
!pip install -q findspark

#**## load dataset**

In [0]:
!curl -O http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip
!cd ml-latest-small/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   584k      0  0:00:01  0:00:01 --:--:--  584k
Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


# set environmental varible

In [0]:
import os
import time
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.3-bin-hadoop2.7"
# spark imports
from pyspark.sql import SparkSession
from pyspark.sql.functions import UserDefinedFunction, explode, desc
from pyspark.sql.types import StringType, ArrayType
from pyspark.mllib.recommendation import ALS

# data science imports
import math
import numpy as np
import pandas as pd

# visualization imports
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# **set path**

## **set session**

In [0]:
!export PATH=$SPARK_HOME/bin:$PATH
!export PYSPARK_PYTHON=python3

In [0]:
# spark config
spark = SparkSession \
    .builder \
    .appName("bisect_l") \
    .config("spark.driver.maxResultSize", "96g") \
    .config("spark.driver.memory", "96g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.master", "local[12]") \
    .getOrCreate()
# get spark context
sc = spark.sparkContext

## **read data**

In [0]:
movies = spark.read.load(os.path.join('/content/ml-latest-small', 'movies.csv'), format='csv', header=True, inferSchema=True)
ratings = spark.read.load(os.path.join('/content/ml-latest-small', 'ratings.csv'),format='csv', header=True, inferSchema=True)
links = spark.read.load(os.path.join('/content/ml-latest-small','links.csv'), format='csv', header=True, inferSchema=True)
tags = spark.read.load(os.path.join('/content/ml-latest-small','tags.csv'), format='csv', header=True, inferSchema=True)

### **load data**

In [0]:
# load data
movie_rating = sc.textFile(os.path.join('/content/ml-latest-small', 'ratings.csv'))
# preprocess data -- only need ["userId", "movieId", "rating"]
header = movie_rating.take(1)[0]
rating_data = movie_rating \
    .filter(lambda line: line!=header) \
    .map(lambda line: line.split(",")) \
    .map(lambda tokens: (int(tokens[0]), int(tokens[1]), float(tokens[2]))) \
    .cache()
# check three rows
rating_data.take(3)

[(1, 1, 4.0), (1, 3, 4.0), (1, 6, 4.0)]

## **split data into train ,test and validate**

In [0]:
train, validation, test = rating_data.randomSplit([7, 1, 2], seed=99)
# cache data
train.cache()
validation.cache()
test.cache()

PythonRDD[49] at RDD at PythonRDD.scala:53

# **Bisect K-Means**


In [0]:
from numpy import array
# $example off$

from pyspark.mllib.clustering import BisectingKMeans

from pyspark.ml.evaluation import ClusteringEvaluator
# $example off$
    # Build the model (cluster the data)
model = BisectingKMeans.train(rating_data, 3, maxIterations=10)
predictions = model.predict(rating_data)
print(predictions)

        # Evaluate clustering
cost = model.computeCost(rating_data)
print("Bisecting K-means Cost = " + str(cost))
    # $example off$ 


 

MapPartitionsRDD[1222] at mapPartitions at PythonMLLibAPI.scala:1336
Bisecting K-means Cost = 21163097895341.08


# K-means 

In [0]:
from math import sqrt

from pyspark.mllib.clustering import KMeans, KMeansModel


# Build the model (cluster the data)
clusters = KMeans.train(rating_data, 3, maxIterations=10, initializationMode="random")

# Evaluate clustering by computing Within Set Sum of Squared Errors
def error(point):
    center = clusters.centers[clusters.predict(point)]
    return sqrt(sum([x**2 for x in (point - center)]))

WSSSE = rating_data.map(lambda point: error(point)).reduce(lambda x, y: x + y)
print("Within Set Sum of Squared Error = " + str(WSSSE))

model = KMeans.train(rating_data, 2)
print("Final centers: " + str(model.clusterCenters))
print("Total Cost: " + str(model.computeCost(rating_data)))

# Save and load model
#clusters.save(sc, "bisect_l/KMeansModel")
#sameModel = KMeansModel.load(sc, "bisect_l//KMeansModel")


Within Set Sum of Squared Error = 516417653.21726453
Final centers: [array([3.26031835e+02, 4.57665585e+03, 3.49835021e+00]), array([3.26565201e+02, 8.73640251e+04, 3.51621726e+00])]
Total Cost: 25525963320001.617
